In [2]:
import os
from glob import glob as g
import pandas as pd
import numpy as np

In [3]:
from rake_nltk import Rake
import nltk
#nltk.download('stopwords')
# Initialize RAKE by optionally providing a list of stopwords
rake = Rake()

In [4]:
mapping_lens = pd.read_csv("X Surg Mapping lens.csv")

In [5]:
for col in mapping_lens.select_dtypes(include=['object', 'category']).columns:
    mapping_lens[col] = mapping_lens[col].fillna('UNK')

In [8]:
mapping_lens

,Unnamed: 0,Index,Detailed_Product,Detailed_Product_EN,Importer,Importer_Standardised,Exporter,Exporter_Standardised,Python_Mapped_Device,Python_Mapped_Family_Name,...,ORIGIN_COUNTRY,Operating Unit,Sub-Operating Unit,Product Type,Company,Family name,Keyword Suggested,Keyword QC,file,Tagged
0,0,594200,"""CEMENTOS PARA HUESOS: CEMENTOS PARA HUESOS (I...","""""""cements for bones: cements for bones (ingen...",QUALITY-1 EXPORT INC,UNK,UNK,UNK,UNK,UNK,...,UNK,Cranial & Spinal Technologies,Biologics,Synthetics,Zimmer Biomet,ingenios,"(ingenios ha,). zimmer biomet",importer,FY24 Q2_Costa Rica_Import_Core Spine_Biologics...,CR FY24Q2
1,1,556866,"""CEMENTOS PARA HUESOS: CEMENTOS PARA HUESOS, U...","""""""cements for bones: cements for bones, medic...",QUALITY-1 EXPORT INC,UNK,UNK,UNK,UNK,UNK,...,UNK,Cranial & Spinal Technologies,Biologics,Synthetics,Zimmer Biomet,CopiOs Cancellous Particulate Xenograft,cements for bones,importer,FY24 Q2_Costa Rica_Import_Core Spine_Biologics...,CR FY24Q2
2,2,556867,"""CEMENTOS PARA HUESOS: CEMENTOS PARA HUESOS, U...","""""""cements for bones: cements for bones, medic...",QUALITY-1 EXPORT INC,UNK,UNK,UNK,UNK,UNK,...,UNK,Cranial & Spinal Technologies,Biologics,Synthetics,Zimmer Biomet,CopiOs Cancellous Particulate Xenograft,cements for bones,importer,FY24 Q2_Costa Rica_Import_Core Spine_Biologics...,CR FY24Q2
3,3,556868,"""CEMENTOS PARA HUESOS: CEMENTOS PARA HUESOS, U...","""""""cements for bones: cements for bones, medic...",QUALITY-1 EXPORT INC,UNK,UNK,UNK,UNK,UNK,...,UNK,Cranial & Spinal Technologies,Biologics,Synthetics,Zimmer Biomet,CopiOs Cancellous Particulate Xenograft,cements for bones,importer,FY24 Q2_Costa Rica_Import_Core Spine_Biologics...,CR FY24Q2
4,4,591182,"""DISPOSITIVO MEDICO DESVIADOR DE FLUJO DE ANEU...","""""""fred intracranial aneurysm flow diverter me...",MICROVENTION COSTA RICA SRL,UNK,UNK,UNK,UNK,UNK,...,UNK,Neurovascular,Neurovascular,Flow Diverter,Terumo,Fred,fred intracranial aneurysm flow diverter,UNK,FY24 Q2_Costa Rica_Import_Core Spine_Biologics...,CR FY24Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17859,6526,1019777,(instrument tracker 9733533xom ent 1pk) acceso...,(instrument tracker 9733533xom ent 1pk) access...,boston scientific de mexico s a de c v,UNK,boston scientific clonmel ltd,UNK,ENT_Core Navigation - Disposable,Medtronic,...,UNK,ENT,ENT,Core Navigation - Disposable,Medtronic,UNK,xom ent,UNK,UNK,MX FY24Q1
17860,6527,755222,(instrument tracker 9733533xom ent 1pk) acceso...,(instrument tracker 9733533xom ent 1pk) access...,boston scientific de mexico s.a. de c.v.,UNK,UNK,UNK,ENT_Core Navigation - Disposable,Medtronic,...,UNK,ENT,ENT,Core Navigation - Disposable,Medtronic,UNK,xom ent,UNK,UNK,MX FY24Q1
17861,6528,949754,(instrument tracker 9733533xom ent 1pk) acceso...,(instrument tracker 9733533xom ent 1pk) access...,medtronic srl cv,UNK,UNK,UNK,ENT_Core Navigation - Disposable,Medtronic,...,UNK,ENT,ENT,Core Navigation - Disposable,Medtronic,UNK,xom ent,UNK,UNK,MX FY24Q1
17862,6529,949753,(instrument tracker 9733533xom ent 1pk) acceso...,(instrument tracker 9733533xom ent 1pk) access...,american eagle mexico imports s de rl de cv,UNK,UNK,UNK,ENT_Core Navigation - Disposable,Medtronic,...,UNK,ENT,ENT,Core Navigation - Disposable,Medtronic,UNK,xom ent,UNK,UNK,MX FY24Q1


In [9]:
for row in mapping_lens.iterrows():
    print("#######")
   
    
    desc = row['Detailed_Product_EN']
    imp = row['Importer']
    exp = row['Exporter']
    identification_soup = "; ".join([desc,imp,exp]) 
    print(identification_soup)
    
    rake.extract_keywords_from_text(identification_soup)

    # Get ranked phrases and scores
    keywords_with_scores = rake.get_ranked_phrases_with_scores()

    # Display the keywords with their scores
    for score, keyword in keywords_with_scores:
        print(f"{keyword}: {score}")
        
    print(row['Keyword Suggested'])
    print()

#######


TypeError: tuple indices must be integers or slices, not str

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV

for col in mapping_lens.columns:
    mapping_lens[col] = mapping_lens[col].astype(str)

from custom_transforms import pass_through, convert_to_string
# Pass through 'keyword_present' without any transformation
pass_through_transformer = FunctionTransformer(pass_through, validate=False)


targets = ['Operating Unit', 'Sub-Operating Unit', 'Product Type', 'Company','Family name']
#Combine categorization columns into a single label
mapping_lens['combined_label'] = mapping_lens[targets].apply(lambda x: '>'.join(x), axis=1)

# Feature engineering.
mapping_lens['augmented_description'] = mapping_lens['Detailed_Product_EN'] + ' KEYWORD_TOKEN ' + mapping_lens['Keyword Suggested']
# Creating a binary feature for keyword presence
mapping_lens['keyword_present'] = mapping_lens.apply(lambda x: x['Keyword Suggested'] in x['Detailed_Product_EN'], axis=1).astype(int)

features = [
    'augmented_description',# Use augmented_description instead of Detailed_Product_EN
    'Importer', 
    'Exporter',
    'Python_Mapped_Device', 
    'Python_Mapped_Family_Name', 
    'Python_Mapped_Manufacturer_Name',
    'keyword_present'  # Make sure this is included
]

target = 'combined_label'





# Define the preprocessing for text columns with custom conversion
# Enhanced text preprocessing pipeline with adjusted TfidfVectorizer parameters
text_preprocessing = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='UNK')),  # Handle missing values
    ('tfidf', TfidfVectorizer(preprocessor=convert_to_string, 
                              ngram_range=(1, 2),  # Consider both unigrams and bigrams
                              max_df=0.95,  # Ignore terms that appear in more than 95% of the documents
                              min_df=5,  # Ignore terms that appear in less than 5 documents
                              max_features=10000))  # Limit the number of features to 10,000
])

# Define the preprocessing for categorical columns
categorical_preprocessing = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),  # Impute with a placeholder
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
# preprocessor = ColumnTransformer(transformers=[
#     ('text', TfidfVectorizer(), 'augmented_description'),
#     ('cat', OneHotEncoder(handle_unknown='ignore'), ['Importer', 'Exporter', 'Python_Mapped_Device', 'Python_Mapped_Family_Name', 'Python_Mapped_Manufacturer_Name']),
#     ('binary', pass_through_transformer, ['keyword_present'])
# ])
# Combine preprocessing steps
preprocessor = ColumnTransformer(transformers=[
    ('text', text_preprocessing, ['augmented_description']),  # Use text_preprocessing here
    ('cat', categorical_preprocessing, ['Importer', 'Exporter', 'Python_Mapped_Device', 'Python_Mapped_Family_Name', 'Python_Mapped_Manufacturer_Name']),
    ('binary', pass_through_transformer, ['keyword_present'])
])

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

#Split data
X_train, X_test, y_train, y_test = train_test_split(mapping_lens[features], mapping_lens[target], test_size=0.1, random_state=44)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)


In [8]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8321208729714605


In [1]:
### Lets test the model
from rake_nltk import Rake
import operator


rake = Rake()

# Example description
new_description = "stent coronary"
importer = "KENDALL S A"
exporter = "MEDTRONIC"
# Extract keywords
rake.extract_keywords_from_text(new_description)
keyword_phrases = rake.get_ranked_phrases_with_scores()

 # For simplicity, let's take the highest scored keyword phrase
# Keywords are returned as a list of (phrase, score) tuples
if keyword_phrases:
    keyword_phrase = max(keyword_phrases, key=operator.itemgetter(0))[1]  # [1] to get the phrase, [0] would give the score
else:
    keyword_phrase = ""
print(new_description)
predict_mapping(model, new_description, importer, exporter)
for score, keyword in keyword_phrases:
    print(f"{keyword}: {score}")
    # Create augmented description
    augmented_description = new_description + ' KEYWORD_TOKEN ' + keyword
    predict_mapping(model, augmented_description, importer, importer)

stent coronary


NameError: name 'predict_mapping' is not defined

In [121]:
## Seems like model is working fine.

In [12]:
from joblib import dump

# Specify the path and filename for the model
model_filename = 'xSurgHierarchyClassifier.joblib'

# Save the model
dump(model, model_filename)

['xSurgHierarchyClassifier.joblib']

In [2]:
from joblib import load
# Load the model from the file
model = load('xSurgHierarchyClassifier.joblib')

MemoryError: Unable to allocate 77.0 MiB for an array with shape (10091952,) and data type float64

In [10]:
def predict_mapping(model, augmented_description, importer, exporter):
    print(augmented_description)
    new_data = pd.DataFrame({
        'augmented_description': [augmented_description],
        'Importer': [importer],  # Use 'UNK' or appropriate value for unknowns
        'Exporter': [exporter],  # Use 'UNK' or appropriate value for unknowns
        'Python_Mapped_Device': ['Z-Unclassified_stent'],  # Use 'UNK' or appropriate value for unknowns
        'Python_Mapped_Family_Name': ['UNK'],  # Use 'UNK' or appropriate value for unknowns
        'Python_Mapped_Manufacturer_Name': ['UNK'],  # Use 'UNK' or appropriate value for unknowns
        'keyword_present': [1]  # No keyword suggested
    })

    predicted_label = model.predict(new_data)
    print("Predicted Combined Label:", predicted_label[0])
    # Predict the probabilities
    predicted_probabilities = model.predict_proba(new_data)

    # Assuming your model is a single-output model or you are interested in the probabilities for a specific output
    # For multi-output models, you'll have to handle each output separately

    # Get the class labels from the model
    class_labels = model.classes_

    # Find the index of the highest probability
    max_prob_index = predicted_probabilities[0].argmax()

    # Get the predicted label with the highest probability
    predicted_label = class_labels[max_prob_index]

    # Get the corresponding probability score
    predicted_score = predicted_probabilities[0][max_prob_index]

    print("Predicted Combined Label:", predicted_label)
    print("Confidence Score:", predicted_score)

Predicted Combined Label: Neurovascular>Neurovascular>Accessories - microcatheter>terumo>Headway


Predicted Combined Label: Neurovascular>Neurovascular>Accessories - microcatheter>terumo>Headway
Confidence Score: 0.23


In [34]:

## This is failing due to Memory issues
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter space
param_distributions = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Randomized search with cross-validation
random_search = RandomizedSearchCV(model, param_distributions, n_iter=10, cv=3, scoring='accuracy', random_state=33)
random_search.fit(X_train, y_train)

print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\pipelin

Best parameters: {'classifier__n_estimators': 100, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': None}
Best cross-validation score: 0.8186228774025004


In [32]:
## Fix errors
# Assuming y_train is your target variable for the training dataset
class_counts = y_train.value_counts()

# Filter classes with only one instance
single_member_classes = class_counts[class_counts == 1]

# Display the classes with only one instance
print("Classes with only one instance:", single_member_classes)


Classes with only one instance: Surgical Innovations>MIS Platforms>Endoscope>Medtronic>Medtronic - Others                        1
Peripheral & Endovenous>Peripheral Vascular>Standard PTA Balloons>Abbott Laboratories>Viatrac    1
Coronary & Renal Denervation>Interventional Coronary>Guiding Catheters>Cardinal Health>UNK       1
Surgical Innovations>Surgical Instrumentation & Access>Access Kit>Geyi Medical Instrument>UNK    1
Renal Care Solutions>Chronic Bellco & Tech Services>Chronic Dialyzers>Baxter>Phoenix             1
                                                                                                ..
Cranial & Spinal Technologies>Core Spine>Spinal Fusion Fixation>Spineguard>DSG                   1
Surgical Innovations>Energy-based Devices>Electrosurgical Devices>Valleylab>sse2                 1
Respiratory Intervention>Airway Management>Tracheostomy>ICU Medical>UNK                          1
Renal Care Solutions>Acute Therapies>Acute Machines>Baxter>Prismax           

In [21]:
y_train_filtered = y_train[y_train.isin(single_member_classes.index)]
X_train_filtered = X_train.loc[y_train_filtered.index]

In [22]:
X_train_filtered

,augmented_description,Importer,Exporter,Python_Mapped_Device,Python_Mapped_Family_Name,Python_Mapped_Manufacturer_Name,keyword_present
12537,carotid stent KEYWORD_TOKEN carotid stent,tecnicos medicos s de rl de cv,nan,PI_carotid-artery-stenting-systems,nan,nan,1
11664,single use catheter for angiography KEYWORD_TO...,immi de mexico production s de r l de c v,nan,CV_Unclassified_Angiography,nan,nan,1
17264,nrg transeptal needle KEYWORD_TOKEN nrg transe...,dise&o y desarrollo medico sa de cv,advanced vascular dynamics semler technologies...,EP_Transseptal Needles,nan,nan,1
11106,sofia catheter 125 cm 131 cm KEYWORD_TOKEN cat...,SPINE LASER TECHNOLOGIES S A,MICROVENTION COSTA RICA SRL GROUP,nan,nan,nan,1
15874,curved blades part for endoscopy equipment bra...,Sin Razon Social,nan,nan,nan,nan,1
...,...,...,...,...,...,...,...
15867,phoenix atherectomy system KEYWORD_TOKEN phoen...,MEDSTENT SA DE CV,"LIFETECH SCIENTIFIC (SHENZHEN) CO., LTD",PI_atherectomy-devices,Phoenix Atherectomy System 2.2 mm Tracking,Philips,1
11131,implantation (stent prosthesis) KEYWORD_TOKEN ...,SERMED & TECH S.A.,TAEWOONG MEDICAL,nan,nan,nan,1
10724,sutures KEYWORD_TOKEN suture,kendall s a,medtronic,Open-Wound Closure_Unclassified_Sutures,nan,nan,1
11746,guide for catheter KEYWORD_TOKEN guide for cat...,cardinal health mexico 244 s de r l de c v,nan,Z-Unclassified CVG_catheter,nan,nan,1


In [15]:
X_train[X_train.index.isin(single_member_classes.index)]

Series([], Name: combined_label, dtype: object)

In [45]:
from sklearn.metrics import classification_report

# Get unique labels from both y_test and y_pred
unique_labels = np.unique(np.concatenate((y_test, y_pred)))

# Generate a classification report
report = classification_report(y_test, y_pred, labels=unique_labels, target_names=unique_labels.astype(str))
print(report)


C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AbhishekSrivast

                                                                                                                           precision    recall  f1-score   support

               Cardiac Ablation Solutions>Cardiac Ablation Solutions>Conventional Ablation Catheter>Boston Scientific>nan       0.00      0.00      0.00         0
                             Cardiac Ablation Solutions>Cardiac Ablation Solutions>Conventional Ablation Catheter>nan>nan       1.00      1.00      1.00         1
                                        Cardiac Ablation Solutions>Cardiac Ablation Solutions>Diagnostic Catheter>J&J>nan       0.92      1.00      0.96        11
                                     Cardiac Ablation Solutions>Cardiac Ablation Solutions>Long Introducer Sheath>J&J>nan       0.00      0.00      0.00         0
                                 Coronary & Renal Denervation>Interventional Coronary>Adapter>Abbott Laboratories>Copilot       0.00      0.00      0.00         0
                     

C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def multioutput_accuracy(y_true, y_pred):
    # Convert to numpy arrays if they are pandas DataFrames
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.to_numpy()
    if isinstance(y_pred, pd.DataFrame):
        y_pred = y_pred.to_numpy()

    accuracies = [accuracy_score(y_true[:, i], y_pred[:, i]) for i in range(y_true.shape[1])]
    return np.mean(accuracies)

custom_scorer = make_scorer(multioutput_accuracy)


# Perform cross-validation using the custom scorer
scores = cross_val_score(model, mapping_lens[features], mapping_lens[targets], cv=5, scoring=custom_scorer)

print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())


Cross-validation scores: [0.49982609 0.72508711 0.57456446 0.7815331  0.21393728]
Average cross-validation score: 0.5589896076352068


In [38]:
## Lets add SMOTE
## If your dataset has imbalanced classes, consider applying techniques 
# like SMOTE for oversampling the minority class or adjusting 
## class weights in the model.



In [39]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



   --------------------------------------- 257.7/257.7 kB 16.5 MB/s eta 0:00:00
   ---------------------------------------- 302.2/302.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [30]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [31]:

# Define the model pipeline with SMOTE
model_with_smote = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=33)),  # Apply SMOTE
    ('classifier', RandomForestClassifier())
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(mapping_lens[features], mapping_lens[target], test_size=0.2, random_state=33)

# Train the model with SMOTE applied to the training data
model_with_smote.fit(X_train, y_train)

# Predict
y_pred = model_with_smote.predict(X_test)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6